In [ ]:
import pandas as pd
import numpy as np
from string import punctuation
from fuzzywuzzy import fuzz
from nltk import word_tokenize
from stop_words import get_stop_words
from nltk.corpus import stopwords
from string import punctuation 
from tqdm import tqdm 

In [ ]:
df = pd.read_csv('result_tar.gz', compression='gzip', sep='\t', error_bad_lines=False)

In [5]:
df = df.drop(["url", "video_url", "source_name", "author_name"], axis=1)

In [6]:
df = df.dropna()

In [7]:
df = df.drop_duplicates()

In [8]:
df = df.reset_index(drop=True)

In [9]:
df.head(10)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian
0,Сборная Турции продолжает свою дорогу на Чемпи...,1.65,"- 1,5",Олимп,2019-06-02 18:00:00,Международные товарищеские матчи,Турция,Узбекистан
1,Двадцать девятого числа состоится один весьма ...,1.52,"– тотал меньше 2,5",Winline,2019-06-28 23:20:00,Кубок Америки,Колумбия,Чили
2,АЗ Алкмар В первом матче голландский клуб созд...,1.85,АЗ Алкмар забьет в обоих таймах,Лига Ставок,2019-08-15 18:30:00,Лига Европы УЕФА,АЗ Алкмаар,Мариуполь
3,"«Сачхере» — «Абердин» И тот, и другой клуб оде...",1.45,Айнтрахт - 3 и более гола,Лига Ставок,2019-07-25 17:00:00,Лига Европы УЕФА,Флора,Айнтрахт Франкфурт
4,Я предполагаю ничью в этом матче с забитыми мя...,5,Арсенал забьет первым и не выиграет,1xСтавка,2019-05-09 19:00:00,Лига Европы УЕФА,Валенсия,Арсенал
5,Июль подходит к концу. Скоро мадридское дерби ...,1.62,Атлетико забьет 1 или 2 гола: да,Фонбет,2019-07-27 00:06:00,Международный Кубок чемпионов,Реал Мадрид,Атлетико Мадрид
6,"«Ахмат», который доставил много проблем команд...",1.71,Ахмат забьет 1 или 2 гола: да,Пари-Матч,2019-07-14 18:30:00,Премьер-Лига,Ахмат,Краснодар
7,"Никто толком не представляет, в каких составах...",2.45,Бавария забьет в обоих таймах,Лига Ставок,2019-07-18 03:00:00,Международный Кубок чемпионов,Арсенал,Бавария
8,"Матч вроде бы товарищеский, но - это и репутац...",3.25,Барселона забьет в каждом тайме,Лига Ставок,2019-07-23 10:30:00,Клубные товарищеские матчи,Барселона,Челси
9,Беларусь – Эстония: прогноз и ставка на матч 1...,1.9,"Беларусь индивидуальный тотал больше 1,5",Пари-Матч,2019-10-10 16:00:00,Чемпионат Европы. Квалификация,Беларусь,Эстония


In [8]:
stop_words_list = get_stop_words('ru')
stop_words_list2 = stopwords.words('russian')
full_stops = set(stop_words_list + stop_words_list2)

### Весь датасет

### Нечеткий поиск по командам 1

In [8]:
%%time
tokens = []
teams1_match = np.zeros(df.shape[0])
for i, text in enumerate(df['result.tsv'].values):
    txt_tokens = []
    txt_tokens.append(word_tokenize(text))
    for text in txt_tokens:
        for t in text:
            if fuzz.ratio(df['team_of_season_1_name_in_russian'][i], t) > 70:
                teams1_match[i] = 1

Wall time: 2h 35min 19s


In [9]:
df['fuzzy_match_for_teams1'] = teams1_match

In [10]:
all_vals = df['fuzzy_match_for_teams1'].shape[0]
print(all_vals)

702908


In [11]:
success = df[df['fuzzy_match_for_teams1'] == 1].shape[0]
print(success)

330689


In [12]:
fuzzyline_team1 = success / all_vals
print(fuzzyline_team1)

0.47045843837315837


### Нечеткий поиск по командам 2

In [13]:
%%time
tokens = []
teams2_match = np.zeros(df.shape[0])
for i, text in enumerate(df['result.tsv'].values):
    txt_tokens = []
    txt_tokens.append(word_tokenize(text))
    for text in txt_tokens:
        for t in text:
            if fuzz.ratio(df['team_of_season_2_name_in_russian'][i], t) > 70:
                teams2_match[i] = 1

Wall time: 2h 30min 34s


In [14]:
df['fuzzy_match_for_teams2'] = teams2_match

In [27]:
all_vals = df['fuzzy_match_for_teams2'].shape[0]
print(all_vals)

702908


In [28]:
success = df[df['fuzzy_match_for_teams2'] == 1].shape[0]
print(success)

321167


In [29]:
fuzzyline_team2 = success / all_vals

In [30]:
print(fuzzyline_team2)

0.4569118575972958


### Нечеткий поиск по лигам

Без нграммов (попробовать снова)

In [9]:
%%time
tokens = []
leagues_match = np.zeros(df.shape[0])
for i, text in enumerate(df['result.tsv'].values):
    txt_tokens = []
    txt_tokens.append(word_tokenize(text))
    for text in txt_tokens:
        for t in text:
            if t not in full_stops:
                if fuzz.ratio(df['league_name_in_russian'][i], t) > 60:
                    leagues_match[i] = 1

Wall time: 2h 20min 43s


In [10]:
df['fuzzy_match_for_leagues'] = leagues_match

In [11]:
all_vals = df['fuzzy_match_for_leagues'].shape[0]
print(all_vals)
success = df[df['fuzzy_match_for_leagues'] == 1].shape[0]
print(success)
fuzzyline_leagues = success / all_vals
print(fuzzyline_leagues)

702908
88254
0.1255555492326165


С использованием нграммов

In [34]:
%%time 
leagues_match = np.zeros(df.shape[0])
for i, text in tqdm(enumerate(df['result.tsv'].values)):
    n_words_league = len(word_tokenize(df['league_name_in_russian'][i]))
    #print(n_words_league)
    N = n_words_league
    words = word_tokenize(text)
    grams = [words[i:i+N] for i in range(len(words)-N+1)]
    #print(i)
    #print(grams)
    for gram in grams: 
        str1 = " "
        joined_gram = str1.join(gram)
        #print(joined_gram)
        if fuzz.ratio(df['league_name_in_russian'][i], joined_gram) > 60:
            #print(df['league_name_in_russian'][i], joined_gram)
            leagues_match[i] = 1


702908it [3:36:13, 54.18it/s] 


Wall time: 3h 36min 13s


In [35]:
df['fuzzy_match_for_leagues'] = leagues_match

In [36]:
all_vals = df['fuzzy_match_for_leagues'].shape[0]
print(all_vals)
success = df[df['fuzzy_match_for_leagues'] == 1].shape[0]
print(success)
fuzzyline_leagues = success / all_vals
print(fuzzyline_leagues)

702908
128578
0.18292294297404496


### Нечеткий поиск по букмекерам

Вариант без нграммов

In [31]:
%%time
tokens = []
bookmaker_match = np.zeros(df.shape[0])
for i, text in enumerate(df['result.tsv'].values):
    txt_tokens = []
    txt_tokens.append(word_tokenize(text))
    for text in txt_tokens:
        for t in text:
            if fuzz.ratio(df['bookmaker_name'][i], t) > 70:
                bookmaker_match[i] = 1

Wall time: 2h 24min 46s


In [32]:
df['fuzzy_match_for_bookmakers1'] = bookmaker_match

In [23]:
all_vals = df['fuzzy_match_for_bookmakers1'].shape[0]
print(all_vals)
success = df[df['fuzzy_match_for_bookmakers1'] == 1].shape[0]
print(success)
fuzzyline_bookmakers = success / all_vals
print(fuzzyline_bookmakers)

702908
1596
0.002270567414227751


Вариант с нграммами

In [37]:
%%time 
bookmaker_match = np.zeros(df.shape[0])
for i, text in tqdm(enumerate(df['result.tsv'].values)):
    n_words_bookmaker = len(word_tokenize(df['bookmaker_name'][i]))
    #print(n_words_league)
    N = n_words_bookmaker
    words = word_tokenize(text)
    grams = [words[i:i+N] for i in range(len(words)-N+1)]
    #print(i)
    #print(grams)
    for gram in grams: 
        str1 = " "
        joined_gram = str1.join(gram)
        #print(joined_gram)
        if fuzz.ratio(df['bookmaker_name'][i], joined_gram) > 60:
            #print(df['league_name_in_russian'][i], joined_gram)
            bookmaker_match[i] = 1


702908it [2:42:52, 71.92it/s] 


Wall time: 2h 42min 52s


In [38]:
df['fuzzy_match_for_bookmakers2'] = bookmaker_match

In [39]:
all_vals = df['fuzzy_match_for_bookmakers2'].shape[0]
print(all_vals)
success = df[df['fuzzy_match_for_bookmakers2'] == 1].shape[0]
print(success)
fuzzyline_bookmakers = success / all_vals
print(fuzzyline_bookmakers)

702908
60190
0.0856299828711581


### Нечеткий поиск по tip_name

C нграммами

In [40]:
%%time 
tipname_match = np.zeros(df.shape[0])
for i, text in tqdm(enumerate(df['result.tsv'].values)):
    n_words_tipname = len(word_tokenize(df['tip_name'][i]))
    #print(n_words_league)
    N = n_words_tipname
    words = word_tokenize(text)
    grams = [words[i:i+N] for i in range(len(words)-N+1)]
    #print(i)
    #print(grams)
    for gram in grams: 
        str1 = " "
        joined_gram = str1.join(gram)
        #print(joined_gram)
        if fuzz.ratio(df['tip_name'][i], joined_gram) > 60:
            #print(df['league_name_in_russian'][i], joined_gram)
            tipname_match[i] = 1
                    

702908it [3:29:25, 55.94it/s] 


Wall time: 3h 29min 25s


In [41]:
df['fuzzy_match_for_tipname'] = tipname_match

In [42]:
all_vals = df['fuzzy_match_for_tipname'].shape[0]
print(all_vals)
success = df[df['fuzzy_match_for_tipname'] == 1].shape[0]
print(success)
fuzzyline_tipname = success / all_vals
print(fuzzyline_tipname)

702908
55231
0.07857500554837903


### Нечеткий поиск по match_date_time

Без нграммов

In [46]:
%%time
tokens = []
date_match = np.zeros(df.shape[0])
for i, text in enumerate(df['result.tsv'].values):
    txt_tokens = []
    txt_tokens.append(word_tokenize(text))
    for text in txt_tokens:
        for t in text:
            if fuzz.ratio(df['match_date_time'][i], t) > 60:
                date_match[i] = 1
                print(df['match_date_time'][i], t)

2019-07-19 19:00:00 2019-07-19
2019-07-07 22:30:00 2019-07-07
2019-07-07 22:30:00 2019-07-07
2019-07-07 22:30:00 2019-07-07
2019-07-07 22:30:00 2019-07-07
2019-07-07 22:30:00 2019-07-07
2019-07-07 22:30:00 2019-07-07
2019-07-07 22:30:00 2019-07-07
2019-07-07 22:30:00 2019-07-07
2019-07-07 22:30:00 2019-07-09
2019-07-07 22:30:00 2019-07-07
2019-07-07 22:30:00 2019-07-07
2019-03-14 17:55:00 2019-03-14
2019-03-14 17:55:00 2019-03-14
2019-03-14 17:55:00 2019-03-14
2019-03-14 17:55:00 2019-03-14
2019-03-07 20:00:00 2019-03-07
2019-04-07 12:00:00 2019-04-07
2019-04-07 12:00:00 2019-04-07
2019-04-07 12:00:00 2019-04-07
2019-04-07 12:00:00 2019-04-07
2019-04-07 12:00:00 2019-04-07
2019-04-07 12:00:00 2019-04-07
2019-04-07 12:00:00 2019-04-07
2019-04-07 12:00:00 2019-04-07
2019-04-07 12:00:00 2019-04-07
2019-04-07 12:00:00 2019-04-07
2019-04-07 12:00:00 2019-04-07
2019-04-07 12:00:00 2019-04-07
2019-04-07 12:00:00 2019-04-07
2019-04-07 12:00:00 2019-04-07
2019-09-05 18:45:00 2019-09-05
2019-05-

In [47]:
df['fuzzy_match_for_date'] = date_match

In [48]:
all_vals = df['fuzzy_match_for_date'].shape[0]
print(all_vals)
success = df[df['fuzzy_match_for_date'] == 1].shape[0]
print(success)
fuzzyline_date = success / all_vals
print(fuzzyline_date)

702908
23
3.272120960353275e-05


### Нечеткий поиск по коэффициентам

Без нграммов

In [ ]:
%%time
tokens = []
coef_match = np.zeros(df.shape[0])
for i, text in enumerate(df['result.tsv'].values):
    txt_tokens = []
    txt_tokens.append(word_tokenize(text))
    for text in txt_tokens:
        for t in text:
            if fuzz.ratio(df['odd'][i], t) > 60:
                coef_match[i] = 1
                #print(df['match_date_time'][i], t)

In [50]:
df['fuzzy_match_for_odd'] = coef_match

In [51]:
all_vals = df['fuzzy_match_for_odd'].shape[0]
print(all_vals)
success = df[df['fuzzy_match_for_odd'] == 1].shape[0]
print(success)
fuzzyline_odd = success / all_vals
print(fuzzyline_odd)

702908
3837
0.005458751358641529


# Русский 

In [10]:
import re
def which_language (text):
    if bool(re.search('[\u0400-\u04FF]', text)) == True:
        return 'russian'
    else:
        return 'english'

In [11]:
lang=[]
for i,text in enumerate(df['result.tsv'].values): 
    lang.append(which_language(str(text)))

In [12]:
df['Language'] = lang

In [13]:
grouped=df.groupby('Language')

In [14]:
for name, group in grouped: 
    if name == 'english': 
        english = group.drop('Language',axis=1) 
    else: 
        russian = group.drop('Language',axis=1)

### команда 1

In [22]:
russian['result.tsv'][29]

'Стабильности «Сетубала» можно позавидовать. Команда из года в год занимает место с 12-го по 15-е и не вылетает из высшей лиги Португалии. Количество очков тоже всегда примерно одинаковое, 30-40.\xa0 В новом сезоне «Витория» набрала один балл после двух туров и расположилась в строчке от вылета. Команда сыграла 0:0 дома с «Тонделой» и была разгромлена «Порту» (0:4).\xa0 В прошлом сезоне команда сумела сделать качественный рывок. Если раньше «Морейренсе» был соседом «Сетубала» по турнирной таблице, то в сезоне 2018/19 команда сумела подняться на шестую строчку.\xa0 Это, конечно, стало поводом заработать на продаже лидеров. За 3.5 миллиона евро в «Бенфику» перешел атакующий полузащитник Шикинью, который поучаствовал почти в половине голов команды в прошлом сезоне. Футболистом интересовались и в чемпионате России.\xa0  В новом чемпионате «Морейренсе» стартовал с поражения от «Браги» (1:3) и разгромной победы над «Жил Висенте» (3:0).\xa0 Искрометного футбола в матче «Витория Сетубал» - «Мо

In [41]:
%%time
tokens = []
teams1_match = np.zeros(russian.shape[0])
for i, text in enumerate(russian['result.tsv']):
    txt_tokens = []
    txt_tokens.append(word_tokenize(text))
    for text in txt_tokens:
        for t in text:
            #print(t)
            try:
                if fuzz.ratio(russian['team_of_season_1_name_in_russian'][i], t) > 65:
#                     print(t, russian['team_of_season_1_name_in_russian'][i])
                    teams1_match[i] = 1
            except:
                continue

Wall time: 2h 42min 6s


In [42]:
russian['fuzzy_match_for_teams1'] = teams1_match

In [43]:
all_vals = russian['fuzzy_match_for_teams1'].shape[0]
print(all_vals)

595931


In [44]:
success = russian[russian['fuzzy_match_for_teams1'] == 1].shape[0]
print(success)

13066


In [45]:
fuzzyline_team1 = success / all_vals
print(fuzzyline_team1)

0.021925357130271793


### команда 2

In [46]:
%%time
tokens = []
teams2_match = np.zeros(russian.shape[0])
for i, text in enumerate(russian['result.tsv']):
    txt_tokens = []
    txt_tokens.append(word_tokenize(text))
    for text in txt_tokens:
        for t in text:
            try:
                if fuzz.ratio(russian['team_of_season_2_name_in_russian'][i], t) > 65:
                    teams2_match[i] = 1
            except:
                continue

Wall time: 2h 12min 42s


In [47]:
russian['fuzzy_match_for_teams2'] = teams2_match

In [48]:
all_vals = russian['fuzzy_match_for_teams2'].shape[0]
print(all_vals)
success = russian[russian['fuzzy_match_for_teams2'] == 1].shape[0]
print(success)
fuzzyline_team2 = success / all_vals
print(fuzzyline_team2)

595931
13617
0.022849960817611436


### Лиги

In [49]:
%%time 
leagues_match = np.zeros(russian.shape[0])
for i, text in tqdm(enumerate(russian['result.tsv'])):
    try:
        n_words_league = len(word_tokenize(russian['league_name_in_russian'][i]))
        #print(n_words_league)
        N = n_words_league
        words = word_tokenize(text)
        grams = [words[i:i+N] for i in range(len(words)-N+1)]
        #print(i)
        #print(grams)
        for gram in grams: 
            str1 = " "
            joined_gram = str1.join(gram)
            #print(joined_gram)
            if fuzz.ratio(russian['league_name_in_russian'][i], joined_gram) > 50:
                #print(df['league_name_in_russian'][i], joined_gram)
                #print(russian['league_name_in_russian'][i], joined_gram)
                leagues_match[i] = 1
    except:
        continue

595931it [2:44:17, 60.45it/s] 


Wall time: 2h 44min 17s


In [50]:
russian['fuzzy_match_for_league'] = leagues_match

In [51]:
all_vals = russian['fuzzy_match_for_league'].shape[0]
print(all_vals)
success = russian[russian['fuzzy_match_for_league'] == 1].shape[0]
print(success)
fuzzyline_league = success / all_vals
print(fuzzyline_league)

595931
64736
0.1086300259593812


In [52]:
russian.to_csv(r'C:\Users\User\Desktop\project\russian_fuzzy.csv', index = False)